## Statoil Iceberg Classification

### Ensemble learning
### Stacking several CNN prediction result for better accuracy

In [3]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

Iceberg_py3_keras.ipynb
Iceberg_py3_keras_angle.ipynb
README.md
ensemble.ipynb
sub.csv
submission.csv
test.json
train.json



In [12]:
sub_path = "./submission"
all_files = os.listdir(sub_path)

outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,0.253535,2.301870e-03,0.036318,0.000943
1,4023181e,0.473720,9.994224e-01,0.995405,0.946086
2,b20200e4,0.304394,7.558256e-22,0.004805,0.006515
3,e7f018bb,0.995573,9.972625e-01,0.997890,0.999929
4,4371c8c3,0.906367,1.000000e+00,0.994407,0.850582


In [13]:
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
is_iceberg_0,1.000000,0.651467,0.583056,0.724461
is_iceberg_1,0.651467,1.000000,0.702787,0.655352
is_iceberg_2,0.583056,0.702787,1.000000,0.790478
is_iceberg_3,0.724461,0.655352,0.790478,1.000000


In [14]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:5].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:5].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:5].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:5].median(axis=1)

In [7]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

### Median Stacking

In [11]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median_2.csv', 
                                        index=False, float_format='%.6f')

### MinMax Median Stacking

In [15]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:5] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:5] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')